In [21]:
import os
import matplotlib.pyplot as plt
import glob
import rasterio
from rasterio.merge import merge
from rasterio.plot import show
from rasterio.mask import mask
from rasterstats import zonal_stats
import fiona
import geopandas as gpd
import xarray as xr
import numpy as np
import rioxarray
import pandas as pd

# GFSAD30AFCE Cropland mosaic

In [ ]:
#all of Africa
GFSAD30AFCE_files = glob.glob('/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/GFSAD30AFCE/*.tif')
GFSAD30AFCE_ea_files = glob.glob('/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/GFSAD30AFCE/EastAfrica/*.tif')

In [ ]:
src_files_to_mosaic = []

In [ ]:
for fp in GFSAD30AFCE_ea_files:
    src = rio.open(fp)
    src_files_to_mosaic.append(src)

In [ ]:
mosaic, out_trans = merge(src_files_to_mosaic)

In [ ]:
out_meta = src.meta.copy()
out_trans
out_meta

In [ ]:
out_meta.update({"height": mosaic.shape[1],
                "width": mosaic.shape[2],
                "transform": out_trans})

In [ ]:
with rio.open('/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/GFSAD30AFCE/Mosaic/GFSAD30AFCE_EA_mosaic.tif', 'w', **out_meta) as dst:
    dst.write(mosaic)

In [ ]:
GFSAD30AFCE = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/GFSAD30AFCE/Mosaic/GFSAD30AFCE_EA_mosaic.tif'
ndvi02071 = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/eMODIS_NDVI/EastAfrica/data.2002.071_mask.tiff'

In [ ]:
fldas = rioxarray.open_rasterio('/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/FLDAS/LIS_HIST_200207010000.d01.nc')
#fldas.rioxarray.reproject("+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs")
fldas.rio.set_crs(4326)

# Create Land Cover Masks

In [23]:
def create_land_mask(in_dir, out_dir, lc_num):
    files=np.array(sorted(os.listdir(in_dir)))
    
    for filename in files:
        base = os.path.splitext(filename)[0]
        print(base)
        rast = rasterio.open(in_dir+filename)
        meta = rast.meta
        rast.close()
        
        with rasterio.open(out_dir+base+'_' + str(lc_num) + '.tif', 'w', **meta) as dst:
            with rasterio.open(in_dir+filename) as src:
                data = src.read()
                data[data!=lc_num] = 0
                data[data==lc_num] = 1
                dst.write(data)

In [24]:
landcover_in_dir = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/MCD12C1/Original/'
landcover_out_dir = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/MCD12C1/Grassland_Mask/'
lc_class = 10

In [25]:
create_land_mask(landcover_in_dir, landcover_out_dir, lc_class)

MCD12C1_T1_2002
MCD12C1_T1_2003
MCD12C1_T1_2004
MCD12C1_T1_2005
MCD12C1_T1_2006
MCD12C1_T1_2007
MCD12C1_T1_2008
MCD12C1_T1_2009
MCD12C1_T1_2010
MCD12C1_T1_2011
MCD12C1_T1_2012
MCD12C1_T1_2013
MCD12C1_T1_2014
MCD12C1_T1_2015
MCD12C1_T1_2016


In [27]:
grassland_mask_dir = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/MCD12C1/Grassland_Mask/'
cropland_mask_dir = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/MCD12C1/Cropland_Mask/'
all_dirs = ['/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/CHIRPS/Resampled/', '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/CHIRTS/Dekads/', '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/eMODIS_NDVI/Resampled/', '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/FLDAS_SM/Dekads/', '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/Hobbins_ET/Resampled/', '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/LST/Resampled/']
grassland_masked_out_dir = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/Variables_Grasslands/'
cropland_masked_out_dir = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/Variables_Croplands/'

In [30]:
def rastercalc_lcmask(lcmask_dir, param_dirs, out_dir):
    lcfiles=np.array(sorted(os.listdir(lcmask_dir)))
    
    
    for yr in range(2002,2017):
        lc_yr = [file for file in lcfiles if str(yr) in file]
        for lc in lc_yr:
            lc_mask = rasterio.open(lcmask_dir + lc)
            for folder in all_dirs:
                var_yr = [file for file in sorted(os.listdir(folder)) if str(yr) in file]
                for var in var_yr:
                    var_rast = rasterio.open(folder+var)
                    meta = var_rast.meta
                    var_lcmask = var_rast.read(1)*lc_mask.read(1)
                    with rasterio.open(out_dir+'cropland_'+ var, 'w', **meta) as dst:
                            dst.write(var_lcmask, 1)

In [31]:
rastercalc_lcmask(grassland_mask_dir, all_dirs, grassland_masked_out_dir)

# Rasterstats with Admin Boundaries

In [32]:
def zone_stat(raster, band, polygon, stats):
    """
    This function will calculate the zonal stats for each polygon within a raster
    requires gpd_df, raster, object and nodata value
    
    Args: raster = input raster
          band = band of raster
          polygon = polygons to calc zonal stats 
          stats = stat to calculate as string
    """
    
    band = raster.read(band)
    zone_stat = zonal_stats(polygon, band, affine=raster.meta['transform'], nodata = 0, stats = stats)
    return zone_stat

In [33]:
def var_poly_join(in_dir, shapefile):
    files=np.array(sorted(os.listdir(in_dir)))
    tifs = pd.Series(files).str.contains('.tif')
    files = files[tifs]
    
    for filename in files:
        print(filename)
        raster = rasterio.open(in_dir+filename)
        stats = zone_stat(raster, 1, shapefile, 'mean')
        name = os.path.splitext(os.path.basename(filename))[0]
        shapefile['Mean'+ "_" + name] = gpd.GeoDataFrame.from_dict(stats)
    return shapefile

In [34]:
adminbds = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/AdminBoundaries/Africa_zones_2019/g2008_af_1.shp'
bndry = gpd.read_file(adminbds)
#bndry.plot()
#oromia = bndry[bndry.ADM1_NAME == 'Oromia']
#oromia.plot()
ea_adminbds = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/AdminBoundaries/gadm36_EastAfrica.shp'
ea_bndry = gpd.read_file(ea_adminbds)

In [88]:
var_grass_dir = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/Variables_Grasslands/'
year_dir = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/Variables_Grasslands/2013/'

In [89]:
output_shp = var_poly_join(year_dir, bndry)

cropland_SM_201301_01.tif
cropland_SM_201301_02.tif
cropland_SM_201301_03.tif
cropland_SM_201302_01.tif
cropland_SM_201302_02.tif
cropland_SM_201302_03.tif
cropland_SM_201303_01.tif
cropland_SM_201303_02.tif
cropland_SM_201303_03.tif
cropland_SM_201304_01.tif
cropland_SM_201304_02.tif
cropland_SM_201304_03.tif
cropland_SM_201305_01.tif
cropland_SM_201305_02.tif
cropland_SM_201305_03.tif
cropland_SM_201306_01.tif
cropland_SM_201306_02.tif
cropland_SM_201306_03.tif
cropland_SM_201307_01.tif
cropland_SM_201307_02.tif
cropland_SM_201307_03.tif
cropland_SM_201308_01.tif
cropland_SM_201308_02.tif
cropland_SM_201308_03.tif
cropland_SM_201309_01.tif
cropland_SM_201309_02.tif
cropland_SM_201309_03.tif
cropland_SM_201310_01.tif
cropland_SM_201310_02.tif
cropland_SM_201310_03.tif
cropland_SM_201311_01.tif
cropland_SM_201311_02.tif
cropland_SM_201311_03.tif
cropland_SM_201312_01.tif
cropland_SM_201312_02.tif
cropland_SM_201312_03.tif
cropland_Tmax_201301_01.tif
cropland_Tmax_201301_02.tif
cropland

In [85]:
output_df = pd.DataFrame(output_shp.drop(columns='geometry'))

In [86]:
output_df.head()

,ADM1_CODE,ADM0_CODE,ADM0_NAME,ADM1_NAME,CONTINENT,REGION,ZONE_ID,POLY_ID,POLY_NAME,WS_START,...,Mean_cropland_rs_pet_201227,Mean_cropland_rs_pet_201228,Mean_cropland_rs_pet_201229,Mean_cropland_rs_pet_201230,Mean_cropland_rs_pet_201231,Mean_cropland_rs_pet_201232,Mean_cropland_rs_pet_201233,Mean_cropland_rs_pet_201234,Mean_cropland_rs_pet_201235,Mean_cropland_rs_pet_201236
0,688,248,Tunisia,Bizerte,Africa,Northern Africa,1,248+688,Tunisia+Bizerte,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,688,248,Tunisia,Bizerte,Africa,Northern Africa,2,248+688,Tunisia+Bizerte,1,...,48.540958,35.527546,27.500847,25.641760,22.555140,16.854847,16.362103,17.934362,13.609810,16.251116
2,688,248,Tunisia,Bizerte,Africa,Northern Africa,3,248+688,Tunisia+Bizerte,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,686,248,Tunisia,Beja,Africa,Northern Africa,4,248+686,Tunisia+Beja,1,...,46.242432,35.131767,29.617201,26.971016,23.589983,17.326719,18.435577,19.260384,14.634191,17.688961
4,700,248,Tunisia,Nabeul,Africa,Northern Africa,5,248+700,Tunisia+Nabeul,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [87]:
#output_shp.to_file(var_crops_dir+'africa2002.shp', driver='ESRI Shapefile')
output_df.to_csv(var_grass_dir+'eastafrica_grasslands.csv')